# Fermentation data import and export

## 1. Imports and directory definitions

In [16]:
import pandas as pd # pandas for tabular data manipulation
import numpy as np  # numpy for numerical operations
import glob
import os
from pandas import ExcelWriter
import openpyxl

data_folder = '../../data/'
fieldlist_path = 'fieldlist/fieldlist_v1.xlsx'
od_cal_file = '2021-10-04_OD_calibration_file.xlsx'

## 2. Experient name definition

In [17]:
experiment = 'DDB_PD_126_AMBR'

## 3. Import data

### 3.1 Main cultures metadata

In [18]:
# get the path to experiment in the LIMS file folder
experiment_file = data_folder+'LIMS_data/'+experiment+'_LIMS.xlsx'
# import the data from the main culture sheet and drop any column that has only NaNs
meta_data = pd.read_excel(experiment_file, sheet_name='main culture').dropna(how='all', axis=1)
    
# cultures.index = range(0,len(cultures.index)) # updating the index of the dataframe
meta_data.rename(columns={'Sample ID': 'Parent Culture'}, inplace=True)
meta_data.sort_values('Reactor/Plate/Flask Number',inplace=True)
meta_data = meta_data.reset_index().drop('index',axis=1)
meta_data

,Parent Culture,Base Medium,Container ID (calculated),Container Type,Culture Type,Experiment,Feed Medium #1,Reactor/Plate/Flask Number,Replicate #,Seed,...,pH Control Base Solution,DO Control Setpoint (%),DO Control Cascade Level 1,DO Control Cascade Level 2,DO Control Cascade Level 3,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type
0,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,DDB_BM_016,R05,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R05,1,DDB_PD_126_AMBR_HMP3071-003_2_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
1,DDB_PD_126_AMBR_HMP3071-003_R06_Main_,DDB_BM_016,R06,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R06,1,DDB_PD_126_AMBR_HMP3071-003_2_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
2,DDB_PD_126_AMBR_HMP3071-003_R07_Main_,DDB_BM_016,R07,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R07,1,DDB_PD_126_AMBR_HMP3071-003_2_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
3,DDB_PD_126_AMBR_HMP3071-003_R08_Main_,DDB_BM_016,R08,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R08,1,DDB_PD_126_AMBR_HMP3071-003_2_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
4,DDB_PD_126_AMBR_HMP3071-003_R09_Main_,DDB_BM_016,R09,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R09,1,DDB_PD_126_AMBR_HMP3071-003_2_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
5,DDB_PD_126_AMBR_HMP3071-003_R10_Main_,DDB_BM_016,R10,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R10,1,DDB_PD_126_AMBR_HMP3071-003_2_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
6,DDB_PD_126_AMBR_SDT714-001_R11_Main_,DDB_BM_016,R11,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R11,1,DDB_PD_126_AMBR_SDT714-001_1_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
7,DDB_PD_126_AMBR_SDT714-001_R12_Main_,DDB_BM_016,R12,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R12,1,DDB_PD_126_AMBR_SDT714-001_1_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
8,DDB_PD_126_AMBR_SDT714-001_R13_Main_,DDB_BM_016,R13,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R13,1,DDB_PD_126_AMBR_SDT714-001_1_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air
9,DDB_PD_126_AMBR_SDT714-001_R14_Main_,DDB_BM_016,R14,AMBR 250,Main,DDB_PD_126_AMBR,DDB_FM_016,R14,1,DDB_PD_126_AMBR_SDT714-001_1_Pre-Final_,...,Ammonium hydroxide 8%,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air


### 3.1 Online data

In [4]:
online_data_dir = 'online_data_9s_interpolated/'
csv_prefix = '_online_stacked_9s.csv'

In [5]:
# import fieldlist
fieldlist = pd.read_excel(fieldlist_path)
# initialize an empty DataFrame with columns from the variables specified in the fieldlist
online_data = pd.DataFrame(columns=list(fieldlist.new_variable_name))

In [6]:
ambr_df = pd.read_csv(data_folder+ online_data_dir + str(experiment) + csv_prefix)

for column in list(online_data.columns): # iterate over the columns of the temporary dataframe. The columns are the variable names of the online data.
    # match the column name (new_variable_name) to the name given by ambr250. Then assign the ambr250 online data to the matching column in the temp_df
    online_data[column] = ambr_df[fieldlist[fieldlist.new_variable_name == column].AMBR_name.iloc[0]]

# adding a column with the experiment name
online_data['Experiment'] = experiment

# Converting the bioreactor naming format in the temporary ambr df
for i in range(1,10):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R0'+str(i))
for i in range(10,25):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R'+str(i))

# adding a column with the Reactor Number
online_data['Reactor/Plate/Flask Number'] = ambr_df['Batch ID'] # adding a column with the reactor ID
    
# dropping some unnecessary rows from the online data DataFrame:
online_data.dropna(axis=0, subset=['Inlet_air_O2'],inplace=True)
    
# updating index
online_data.index = range(0,len(online_data.index)) # updating the index of the dataframe
    
# Merging online data with cultures metadata
online_data = pd.merge(left=online_data,right=meta_data)
    
# phase annotation
feed_vol_list = online_data['Feed_volume'].tolist()
phase_list = ['feed' if i > 0.0 else 'batch' for i in feed_vol_list]
online_data['Phase'] = phase_list
online_data.head()

,Time,DO,pH,Stirrer_speed,Temperature,Pressure,Air_flow,Oxygen_flow,Inlet_air_O2,Offgas_O2,...,DO Control Setpoint (%),DO Control Cascade Level 1,DO Control Cascade Level 2,DO Control Cascade Level 3,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type,Phase
0,0.0000,98.003187,6.531001,999.001076,29.620008,2.566943,100.105940,0.0,20.95,20.702000,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch
1,0.0025,97.769089,6.531478,999.090222,29.614775,2.361823,100.118645,0.0,20.95,20.702433,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch
2,0.0050,97.536931,6.532000,997.449997,29.622250,2.354350,100.075752,0.0,20.95,20.704450,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch
3,0.0075,97.385832,6.532000,998.951097,29.629755,2.508963,99.828069,0.0,20.95,20.705950,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch
4,0.0100,97.236052,6.532000,1001.164094,29.644427,2.479375,100.064540,0.0,20.95,20.706722,...,40,Stirring,Aeration,Oxygen,0.1,0.25,1000,4000,Air,batch


### 3.2 Offline data

#### 3.2.1 Timepoint data

In [7]:
timepoint_cols = ['Entity','Experiment','Parent Culture','Reactor/Plate Number','Timepoint (#)','Timepoint (h)','Volume','OD']
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experimentin the LIMS file folder

timepoint_data = pd.read_excel(file, sheet_name='timepoint samples')[timepoint_cols] # importing timepoint data from one experiment into a temporary DFF

# renaming some of the columns in the dataframe and reindexing
timepoint_data.rename(columns={"Entity": "Timepoint Sample", "Reactor/Plate Number": "Reactor/Plate/Flask Number", "Volume": "Sample volume (mL)"},inplace=True)
timepoint_data = timepoint_data.reset_index().drop('index',axis=1)
timepoint_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD
0,DDB_PD_126_AMBR_R05__S01,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S01,12.244256,7.0,4.575
1,DDB_PD_126_AMBR_R05__S02,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S02,18.887388,5.0,29.180
2,DDB_PD_126_AMBR_R05__S03,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S03,38.929876,5.5,68.250
3,DDB_PD_126_AMBR_R05__S04,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S04,58.927792,5.5,116.300
4,DDB_PD_126_AMBR_R05__S05,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S05,78.927893,7.5,184.900


#### 3.2.2 Analytical data

In [8]:
analytical_cols = ['Name', 'Analysis Type', 'Cellular Compartment', 'Dilution Factor','Experiment', 'Parent Culture', 'Sample Prep Date', 'Timepoint (h)', 'Timepoint Sample', 'compound_name', 'concentration','concentration_unit']
    
analytical_data = pd.DataFrame(columns=analytical_cols) # initializing an empty DF for timepoint_data
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experiment in the LIMS file folder

analytical_data = pd.read_excel(file, sheet_name='analytical samples')[analytical_cols] # importing timepoint data from one experiment into a temporary DF

# renaming one of the columns in the dataframe and reindexing
analytical_data.rename(columns={"Name": "Analytical Sample"},inplace=True)
analytical_data = analytical_data.reset_index().drop('index',axis=1)

# correcting for the dilution factor, then dropping the dilution factor
analytical_data['concentration'] = analytical_data['concentration'] * analytical_data['Dilution Factor']
analytical_data.drop(['Dilution Factor'], axis=1, inplace=True)
# removing any analytical data not obtained from a main culture sample
analytical_data = analytical_data[analytical_data['Parent Culture'].notnull()]
analytical_data.head()

,Analytical Sample,Analysis Type,Cellular Compartment,Experiment,Parent Culture,Sample Prep Date,Timepoint (h),Timepoint Sample,compound_name,concentration,concentration_unit
0,DDB_PD_126_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,2023-05-09,12.244256,DDB_PD_126_AMBR_R05__S01,citric acid,0.194946,g/L
1,DDB_PD_126_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,2023-05-09,12.244256,DDB_PD_126_AMBR_R05__S01,acetic acid,0.349354,g/L
2,DDB_PD_126_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,2023-05-09,12.244256,DDB_PD_126_AMBR_R05__S01,lactose,0.000000,g/L
3,DDB_PD_126_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,2023-05-09,12.244256,DDB_PD_126_AMBR_R05__S01,pyruvic acid,0.000000,g/L
4,DDB_PD_126_AMBR_R05__S01_SOA_#1,SOA,Extracellular Region,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,2023-05-09,12.244256,DDB_PD_126_AMBR_R05__S01,potassium dihydrogen phosphate,9.759030,g/L


In [9]:
analytical_data['Analysis Type'].unique()

array(['SOA', 'TM'], dtype=object)

##### 3.2.2.1 Pivot analytical data

In [10]:
analytical_data['compound_name_and_unit'] = analytical_data['compound_name'] + ' [' + analytical_data['concentration_unit'] + ']'

analytical_data_pivot = analytical_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
analytical_data_pivot['Timepoint Sample'] = analytical_data_pivot.index

analytical_data_pivot.index.name = None
analytical_data_pivot.columns.name = None
analytical_data_pivot = analytical_data_pivot.reset_index().drop('index',axis=1)

analytical_data_pivot.head()

,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],lactose [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L],tryptophan [g/L],Timepoint Sample
0,13.252213,0.349354,0.194946,0.0,0.000000,0.000000,0.0,9.759030,0.0,0.000000,0.000000,DDB_PD_126_AMBR_R05__S01
1,0.000000,0.085708,0.197249,0.0,0.000000,0.000000,0.0,8.373409,0.0,0.000000,0.264018,DDB_PD_126_AMBR_R05__S02
2,0.000000,0.080670,0.247385,0.0,0.000000,0.000000,0.0,9.147414,0.0,0.045158,0.917556,DDB_PD_126_AMBR_R05__S03
3,0.000000,0.461899,0.300133,0.0,0.215957,0.062034,0.0,7.413821,0.0,0.159813,1.065042,DDB_PD_126_AMBR_R05__S04
4,0.000000,0.659381,0.303265,0.0,0.355752,0.000000,0.0,4.457014,0.0,0.260247,1.371156,DDB_PD_126_AMBR_R05__S05


#### 3.2.3 Trace_metal_data

In [11]:
trace_metal_data = pd.read_excel(data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx', sheet_name='trace metal export')
trace_metal_data.rename(columns={'Name': 'Analytical Sample'}, inplace=True)

# timepoint_samples_temp = []
# for i in df_tm['Analytical Sample'].to_list():
#     timepoint_samples_temp.append(i.split('_TM')[0])

# df_tm['Timepoint Sample'] = timepoint_samples_temp
trace_metal_data['Timepoint Sample'] = trace_metal_data['Analytical Sample'].apply(lambda x: pd.Series(str(x).split("_TM")[0]))
trace_metal_data.head()

,Analytical Sample,compound_name,concentration,concentration_unit,Timepoint Sample


##### 3.2.2.1 Pivot trace metal data

In [12]:
trace_metal_data['compound_name_and_unit'] = trace_metal_data['compound_name'] + ' [' + trace_metal_data['concentration_unit'] + ']'

trace_metal_data_pivot = trace_metal_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
trace_metal_data_pivot['Timepoint Sample'] = trace_metal_data_pivot.index

trace_metal_data_pivot.index.name = None
trace_metal_data_pivot.columns.name = None
trace_metal_data_pivot = trace_metal_data_pivot.reset_index().drop('index',axis=1)

trace_metal_data_pivot.head()

,Timepoint Sample


#### 3.2.4 Custom data

In [13]:
try:
    custom_data = pd.read_excel(data_folder+'/custom_data_import_files/'+experiment+'_custom.xlsx')
    custom_data.head()
except:
    custom_data = pd.DataFrame(columns=['Timepoint Sample'])
    
custom_data

,Timepoint Sample,CDW_1 [g/L],CDW_2 [g/L],CDW_3 [g/L]
0,DDB_PD_126_AMBR_R05__S01,1.900000,2.200000,1.800000
1,DDB_PD_126_AMBR_R05__S02,7.500000,7.100000,7.900000
2,DDB_PD_126_AMBR_R05__S03,15.000000,16.000000,16.000000
3,DDB_PD_126_AMBR_R05__S04,32.000000,31.900000,31.600000
4,DDB_PD_126_AMBR_R05__S05,50.600000,52.400000,52.100000
5,DDB_PD_126_AMBR_R06__S01,1.800000,1.700000,0.700000
6,DDB_PD_126_AMBR_R06__S02,7.500000,7.400000,6.500000
7,DDB_PD_126_AMBR_R06__S03,16.333333,15.833333,11.333333
8,DDB_PD_126_AMBR_R06__S04,31.400000,31.700000,31.700000
9,DDB_PD_126_AMBR_R06__S05,52.300000,52.400000,52.500000


#### 3.2.5 Merging all offline data

In [14]:
ac_merge = pd.merge(left=analytical_data_pivot, right=trace_metal_data_pivot, how='left')
custom_merge = pd.merge(left=ac_merge, right=custom_data, how='left')
offline_data = pd.merge(left=timepoint_data, right=custom_merge, how='left')
offline_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD,D-glucose [g/L],acetic acid [g/L],...,formic acid [g/L],lactate [g/L],lactose [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L],tryptophan [g/L],CDW_1 [g/L],CDW_2 [g/L],CDW_3 [g/L]
0,DDB_PD_126_AMBR_R05__S01,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S01,12.244256,7.0,4.575,13.252213,0.349354,...,0.000000,0.000000,0.0,9.759030,0.0,0.000000,0.000000,1.9,2.2,1.8
1,DDB_PD_126_AMBR_R05__S02,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S02,18.887388,5.0,29.180,0.000000,0.085708,...,0.000000,0.000000,0.0,8.373409,0.0,0.000000,0.264018,7.5,7.1,7.9
2,DDB_PD_126_AMBR_R05__S03,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S03,38.929876,5.5,68.250,0.000000,0.080670,...,0.000000,0.000000,0.0,9.147414,0.0,0.045158,0.917556,15.0,16.0,16.0
3,DDB_PD_126_AMBR_R05__S04,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S04,58.927792,5.5,116.300,0.000000,0.461899,...,0.215957,0.062034,0.0,7.413821,0.0,0.159813,1.065042,32.0,31.9,31.6
4,DDB_PD_126_AMBR_R05__S05,DDB_PD_126_AMBR,DDB_PD_126_AMBR_HMP3071-003_R05_Main_,R05,S05,78.927893,7.5,184.900,0.000000,0.659381,...,0.355752,0.000000,0.0,4.457014,0.0,0.260247,1.371156,50.6,52.4,52.1


## 4. Export data

In [15]:
meta_data.to_csv(experiment + '_meta_data_export.csv',index=False)
offline_data.to_csv(experiment + '_offline_data_export.csv',index=False)
online_data.to_csv(experiment + '_online_data_export.csv',index=False)